In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [196]:
#set up multivariate linear regression example data 

#define the dataset size
num_features = 5
num_samples = 1000

#define dataset parameters 
model_a = np.random.normal(size=(num_features))
model_c =np.random.normal()

print("model a is {}".format(model_a))
print("model c is {}".format(model_c))

# create some random initial values and then work out the result adding in some random noise
x_train = np.random.normal(size=(num_samples,num_features))*100
y_train = x_train @ model_a + model_c + np.random.normal(size= num_samples)


model a is [-1.11494296 -1.70482218  0.0446201   0.96751824  1.24041509]
model c is -0.7759795581324975


In [199]:
#use tensorflow to compute gradients and optimise the fit

a = tf.Variable(tf.zeros(shape=(num_features,)),name="a", trainable=True, dtype=tf.float32)
c = tf.Variable(tf.zeros(shape=(1,)),name ="c", trainable=True, dtype=tf.float32)

@tf.function
def f(x):
    return tf.tensordot(x,a,axes=1)+c

#print(tf.autograph.to_code(f.python_function))

variables = [a,c]

optimizer = tf.optimizers.Adam(0.005)
loss_object = tf.keras.losses.MeanSquaredError()

for i in range(1000):
    with tf.GradientTape() as tape:
        y_pred = f(tf.cast(x_train,tf.float32))
        loss = loss_object(tf.cast(y_train,tf.float32),y_pred)
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        if i %100 ==0: 
            print('Epoch: {} Loss: {}'.format(i,loss.numpy()))

Epoch: 0 Loss: 69665.5078125
Epoch: 100 Loss: 29789.58984375
Epoch: 200 Loss: 11134.5341796875
Epoch: 300 Loss: 3785.403076171875
Epoch: 400 Loss: 1207.9327392578125
Epoch: 500 Loss: 359.8683166503906
Epoch: 600 Loss: 97.5340576171875
Epoch: 700 Loss: 23.923290252685547
Epoch: 800 Loss: 5.783719539642334
Epoch: 900 Loss: 1.8958250284194946


In [203]:
print('Model fit:',a.numpy(),c.numpy())
print('True :    ',model_a,[model_c])

Model fit: [-1.1156785  -1.7006475   0.04418615  0.96755296  1.2408987 ] [-1.0829409]
True :     [-1.11494296 -1.70482218  0.0446201   0.96751824  1.24041509] [-0.7759795581324975]


In [204]:
# statsmodels OLS .. remember to add in the constant
import statsmodels.api as sm

model = sm.OLS(y_train, sm.add_constant(x_train)).fit()
print(model.summary(),model.params)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.435e+07
Date:                Thu, 08 Oct 2020   Prob (F-statistic):               0.00
Time:                        10:51:54   Log-Likelihood:                -1401.3
No. Observations:                1000   AIC:                             2815.
Df Residuals:                     994   BIC:                             2844.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8157      0.031    -26.094      0.0

In [208]:
#SKlearn linear model

from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(x_train,y_train)
vars(regr)
print('Model fit :',regr.coef_,regr.intercept_)
print('True :     ',model_a,model_c)
vars(regr)

Model fit : [-1.11516753 -1.70416648  0.04430329  0.96727484  1.24074803] -0.8156707622412469
True :      [-1.11494296 -1.70482218  0.0446201   0.96751824  1.24041509] -0.7759795581324975


{'fit_intercept': True,
 'normalize': False,
 'copy_X': True,
 'n_jobs': None,
 'coef_': array([-1.11516753, -1.70416648,  0.04430329,  0.96727484,  1.24074803]),
 '_residues': 965.249797073664,
 'rank_': 5,
 'singular_': array([3352.273302  , 3240.251876  , 3188.43424947, 3034.15967049,
        3009.09722493]),
 'intercept_': -0.8156707622412469}

In [215]:
#optimise loss function. Need to combine a+c into the first parameter 

from scipy.optimize import minimize

def cost(a,x,y):
    predict = a[-1] + x@a[:-1]
    return np.mean((predict-y)**2)

opt_result = minimize(cost, [0,0,0,0,0,0], args=(x_train,y_train),method='BFGS', options={'maxiter': 5000})
print('Model fit :',opt_result.x[:-1],opt_result.x[-1])
print('True :     ',model_a,model_c)


Model fit : [-1.11516754 -1.70416649  0.04430328  0.96727483  1.24074802] -0.8156707985829653
True :      [-1.11494296 -1.70482218  0.0446201   0.96751824  1.24041509] -0.7759795581324975
